# 메타크리틱 점수 수집

## 조건
- 수집 항목 : 메타스코어, 유저스코어
- 수집url : https://www.metacritic.com
- 검색 기준
    - 게임명 전처리(소문자 변환, 영문·숫자만 남김)
    - 검색 결과 1순위 항목과 입력 게임명 일치 여부 확인 후 클릭

In [13]:
import pandas as pd
df = pd.read_csv('02-3. steam_data_real.csv')
df[df['게임명'].str.contains('오버', na=False)]

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
17115,2357570,오버워치™ 2,2023년 08월 11일,무료 플레이 | MOBA | 히어로 슈팅 | 1인칭 슈팅 | 3인칭 슈팅,Windows,0,평가 점수: 대체로 부정적,"사용자 평가 348,551개"
17487,3665200,오버워치 2: 리퍼 완전한 신화 무기 스킨 묶음 상품,2025년 06월 25일,MOBA | 히어로 슈팅 | 3인칭 슈팅 | 1인칭 슈팅 | 1인칭,Windows,0,NaN,NaN


In [18]:
# 게임명 정제: 모든 특수문자 제거 (단, 공백과 한글/영문/숫자만 유지)
df['게임명'] = df['게임명'].str.replace(r'[^\w\s가-힣]', '', regex=True)

# 연속 공백 하나로 
df['게임명'] = df['게임명'].str.replace(r'\s{2,}', ' ', regex=True)

# 공백 -으로 변경
df['게임명'] = df['게임명'].str.replace(' ', '-')

# 예시 결과 확인
df['게임명']

0               guilty-gear-strive
1        magic-the-gathering-arena
2                   ez2on-reboot-r
3               sons-of-the-forest
4                    left-4-dead-2
                   ...            
19421               crabs-and-rats
19422                adventure-jam
19423            the-train-arrived
19424             dispatch-control
19425             watertrix-online
Name: 게임명, Length: 19426, dtype: object

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pickle
import re

# 크롬 브라우저 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

df = pd.read_csv('02-3. steam_data_real.csv')
df['게임명'] = df['게임명'].str.replace(r'[^\w\s가-힣]', '', regex=True)
df['게임명'] = df['게임명'].str.replace(r'\s{2,}', ' ', regex=True)
df['게임명'] = df['게임명'].str.replace(' ', '-')

for i in df['게임명'] :
    url = f'https://www.metacritic.com/game/{i}/'
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('div', 'c-productHero_scoreInfo g-inner-spacing-top-medium g-outer-spacing-bottom-medium g-outer-spacing-top-medium')
    score = data.find_all('div', 'c-productScoreInfo_scoreNumber u-float-right')

meta = []

In [12]:
for i in score :
    print(i.text.strip())

79
1.8


In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pickle
import re

# 크롬 브라우저 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

df = pd.read_csv('02-3. steam_data_real.csv')
df['게임명'] = df['게임명'].str.replace(r'[^\w\s가-힣]', '', regex=True)
df['게임명'] = df['게임명'].str.replace(r'\s{2,}', ' ', regex=True)
df['게임명'] = df['게임명'].str.replace(' ', '-')

for i in df['게임명'] :
    url = f'https://www.metacritic.com/game/{i}/'
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('div', 'c-productHero_scoreInfo g-inner-spacing-top-medium g-outer-spacing-bottom-medium g-outer-spacing-top-medium')
    score = data.find_all('div', 'c-productScoreInfo_scoreNumber u-float-right')

    meta = []
    for i in score : 
        meta.append(i.text.strip())

    game = i
    meta_score = meta[0] if len(meta) > 0 else 'N/A'
    user_score = meta[1] if len(meta) > 1 else 'N/A'

In [2]:
import pandas as pd
df = pd.read_csv('최종_영문게임이름.csv')
df['game_name']

0                GUILTY GEAR -STRIVE-
1          Magic: The Gathering Arena
2                    EZ2ON REBOOT : R
3                  Sons Of The Forest
4                       Left 4 Dead 2
                     ...             
19421          Animals for Clip maker
19422    DJMAX RESPECT V - NEXON PACK
19423           ARK: Survival Evolved
19424                Backroom Company
19425                WATERTRIX ONLINE
Name: game_name, Length: 19426, dtype: object

### 메타크리틱 점수 추출

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException

# -크롬 브라우저 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

def clean_game_name(name):
    name = name.lower()
    name = re.sub(r'[^a-z0-9]', '', name)
    return name

def remove_non_bmp_characters(text):
    return ''.join(c for c in text if ord(c) <= 0xFFFF)

# 데이터 불러오기
df = pd.read_csv('최종_영문게임이름.csv')
df.loc[156, 'game_name'] = 'Wrap House Simulator'
output_csv = 'metacritic_scores.csv'

if os.path.exists(output_csv):
    df_scores = pd.read_csv(output_csv)
    done_games = set(df_scores['게임명'].tolist())
else:
    df_scores = pd.DataFrame(columns=['게임명', 'meta_score', 'user_score'])
    done_games = set()

# -드라이버 시작 함수
def start_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    # 메인 페이지로 이동 (검색 페이지가 아닌)
    driver.get("https://www.metacritic.com/")
    time.sleep(2)
    return driver

# uh-oh 페이지 체크 함수
def is_error_page(driver):
    """uh-oh 페이지나 에러 페이지인지 정확하게 확인"""
    try:
        current_url = driver.current_url.lower()
        page_source = driver.page_source.lower()
        
        # 1. URL 기반 에러 페이지 감지 (가장 확실한 방법)
        if "/error" in current_url or "404" in current_url:
            return True
            
        # 2. 명확한 에러 페이지 텍스트만 확인 (더 엄격하게)
        if "uh-oh" in page_source and "not the page you're looking for" in page_source:
            return True
            
        # 3. 페이지 제목 확인
        try:
            title = driver.title.lower()
            if "error" in title or "not found" in title:
                return True
        except:
            pass
            
        return False
        
    except Exception:
        return False  # 확실하지 않으면 에러로 판단하지 않음

# -메인 페이지로 이동 함수
def go_to_main_page(driver):
    """메인 페이지로 이동"""
    try:
        print("메인 페이지로 이동 중...")
        driver.get("https://www.metacritic.com/")
        time.sleep(3)  # 로딩 시간 증가
        
        # 메인 페이지 로딩 확인
        if "metacritic" in driver.current_url.lower() and not is_error_page(driver):
            print("메인 페이지 로딩 완료")
            return True
        else:
            print("메인 페이지 로딩 실패")
            return False
    except Exception as e:
        print(f"메인 페이지로 이동 중 오류: {e}")
        return False

driver = start_driver()

# 메인 크롤링 루프
for game_name in df['game_name']:
    if game_name in done_games:
        print(f"{game_name} 는 이미 크롤링됨. 건너뜀.")
        continue

    retry_count = 0
    max_retries = 3
    
    while retry_count < max_retries:  # 최대 재시도 횟수 제한
        try:
            # 디버깅: 현재 상태 출력
            print(f"'{game_name}' 처리 시작 - 현재 URL: {driver.current_url}")
            
            # 에러 페이지 체크
            if is_error_page(driver):
                print(f"🔁 '{game_name}' 시작 시 에러 페이지 감지. 다음 게임으로 넘어갑니다.")
                # 메인 페이지로 복구만 하고 이 게임은 건너뜀
                go_to_main_page(driver)
                break

            # 검색 및 전처리
            safe_game_name = remove_non_bmp_characters(game_name)

            # 검색창 찾기 시도
            try:
                search_input = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/header/div[1]/div/div[2]/div/div/form/input')
            except NoSuchElementException:
                print(f"검색창을 찾을 수 없음. 메인 페이지로 이동 후 재시도.")
                if not go_to_main_page(driver):
                    raise Exception("메인 페이지 이동 실패")
                search_input = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/header/div[1]/div/div[2]/div/div/form/input')
            
            search_input.clear()
            search_input.send_keys(game_name)
            time.sleep(1.5)
            search_input.send_keys(Keys.ENTER)
            time.sleep(2)

            # 검색 후 명확한 에러 페이지만 체크
            if is_error_page(driver):
                print(f"검색 후 명확한 에러 페이지 감지. '{game_name}' 건너뜀.")
                go_to_main_page(driver)  # 메인 페이지로 복구
                break

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            result_tag = soup.find('p', class_='g-text-medium-fluid g-text-bold g-outer-spacing-bottom-small u-text-overflow-ellipsis')

            if result_tag:
                name = result_tag.text.strip()
                clean_target = clean_game_name(game_name)
                clean_found = clean_game_name(name)

                if clean_target == clean_found:
                    print(f"'{game_name}' 과 '{name}' 은 같다고 판단되어 클릭합니다.")

                    try:
                        driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/a/div[2]/p').click()
                        time.sleep(2)

                        # 클릭 후 에러 페이지 체크
                        if is_error_page(driver):
                            print(f"❌ '{game_name}' 상세 페이지가 존재하지 않음 (에러 페이지). 다음 게임으로 넘어감.")
                            go_to_main_page(driver)  # 메인 페이지로 복구
                            break

                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')

                        data = soup.find('div', class_='c-productHero_scoreInfo g-inner-spacing-top-medium g-outer-spacing-bottom-medium g-outer-spacing-top-medium')

                        if data:
                            score = data.find_all('div', class_='c-productScoreInfo_scoreNumber u-float-right')
                            meta = [s.text.strip() for s in score]

                            meta_score = meta[0] if len(meta) > 0 else 'N/A'
                            user_score = meta[1] if len(meta) > 1 else 'N/A'
                        else:
                            meta_score = 'N/A'
                            user_score = 'N/A'

                        # 결과 저장
                        df_scores = pd.concat([df_scores, pd.DataFrame([{
                            '게임명': game_name,
                            'meta_score': meta_score,
                            'user_score': user_score
                        }])], ignore_index=True)

                        df_scores.to_csv(output_csv, index=False)
                        print(f"{game_name} 크롤링 완료 및 저장됨. 점수 : {meta_score}, {user_score}")
                        
                    except Exception as e_click:
                        print(f"'{game_name}' 클릭 중 오류 발생: {e_click}")
                        break  # 클릭 실패는 재시도 안 함

                else:
                    print(f"'{game_name}' 과 '{name}' 은 다름. 건너뜀.")
                    go_to_main_page(driver)  # 메인 페이지로 복구
                    break

            else:
                print(f"{game_name} 검색 결과 없음.")
                go_to_main_page(driver)  # 메인 페이지로 복구
                break

            break  # 정상적으로 처리 완료

        except WebDriverException as e:
            error_message = str(e).lower()
            print(f"{game_name} 크롤링 중 WebDriverException 발생 (시도 {retry_count + 1}): {e}")

            if 'time' in error_message or 'readout' in error_message:
                print("⛔ time readout 관련 에러 발생! 브라우저 재시작 후 재시도합니다.")
                try:
                    driver.quit()
                except:
                    pass
                driver = start_driver()
                retry_count += 1
                continue

            else:
                print("⛔ 기타 WebDriverException 발생. 다음 게임으로 넘어갑니다.")
                break

        except NoSuchElementException as e:
            retry_count += 1
            print(f"{game_name} 크롤링 중 NoSuchElementException 발생 (시도 {retry_count}): {e}")
            
            if retry_count < max_retries:
                print("⛔ 브라우저 재시작 후 재시도합니다.")
                try:
                    driver.quit()
                except:
                    pass
                driver = start_driver()
                continue
            else:
                print("⛔ 최대 재시도 횟수 초과. 다음 게임으로 넘어갑니다.")
                break

        except Exception as e:
            print(f"{game_name} 크롤링 중 기타 오류 발생: {e}")
            break  # 기타 오류는 재시도 없이 다음 게임으로 넘어감

    if retry_count >= max_retries:
        print(f"'{game_name}' 최대 재시도 횟수 초과로 건너뜀.")

# 드라이버 종료
try:
    driver.quit()
except:
    pass

In [31]:
df = pd.read_csv('최종_영문게임이름.csv')
df

,appid,game_name
0,1384160,GUILTY GEAR -STRIVE-
1,2141910,Magic: The Gathering Arena
2,1477590,EZ2ON REBOOT : R
3,1326470,Sons Of The Forest
4,550,Left 4 Dead 2
...,...,...
19421,1723880,Animals for Clip maker
19422,1782170,DJMAX RESPECT V - NEXON PACK
19423,346110,ARK: Survival Evolved
19424,3010460,Backroom Company


### 누락된 것 재크롤링

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver():
    """새로운 크롬 브라우저 인스턴스를 생성합니다."""
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_experimental_option('detach', True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

# 브라우저 재시작 함수
def restart_driver(current_driver):
    """현재 브라우저를 종료하고 새로운 브라우저를 시작합니다."""
    try:
        current_driver.quit()
        print("기존 브라우저 종료 완료")
    except:
        print("기존 브라우저 종료 중 오류 발생 (무시)")
    
    time.sleep(2)
    new_driver = init_driver()
    print("새로운 브라우저 시작 완료")
    return new_driver

driver = init_driver()
input_file = '최종_영문게임이름.csv'
save_file = 'metacritic_scores.csv'

# 기존 저장 데이터 불러오기
df_games = pd.read_csv(input_file)

if os.path.exists(save_file) and os.path.getsize(save_file) > 0:
    df_saved = pd.read_csv(save_file)
    if "검색어" in df_saved.columns:
        done_games = set(df_saved["검색어"])
    else:
        done_games = set()
else:
    df_saved = pd.DataFrame(columns=["검색어", "게임명", "메타스코어", "유저스코어", "게임URL"])
    done_games = set()

print(f"이미 처리된 게임 수: {len(done_games)}")

# 문자열 정리 함수
def clean_game_name(name):
    name = name.lower()
    name = re.sub(r'[^a-z0-9]', '', name)
    return name

# -메타크리틱 검색 및 데이터 수집
max_retries = 3  # 최대 재시도 횟수

for game in df_games['game_name']:
    if game in done_games:
        print(f"이미 처리된 게임 → 건너뜀: {game}")
        continue

    print(f"검색 시작: {game}")
    
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            # WebDriverWait 설정 (타임아웃 10초)
            wait = WebDriverWait(driver, 10)
            
            url = 'https://www.metacritic.com/game/'
            driver.get(url)
            time.sleep(0.5)

            # 검색창 찾기 (타임아웃 적용)
            try:
                search_box = wait.until(EC.presence_of_element_located((By.XPATH,
                    '//*[@id="__layout"]/div/div[2]/header/div[1]/div/div[2]/div/div/form/input'
                )))
            except TimeoutException:
                print(f"검색창 못 찾음 → 새 창 열어서 재시도: {game}")
                driver.execute_script("window.open('https://www.metacritic.com/game/');")
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(1)
                try:
                    search_box = wait.until(EC.presence_of_element_located((By.XPATH,
                        '//*[@id="__layout"]/div/div[2]/header/div[1]/div/div[2]/div/div/form/input'
                    )))
                except TimeoutException:
                    raise TimeoutException("검색창을 찾을 수 없습니다.")

            # 검색 실행
            search_box.clear()
            search_box.send_keys(game)
            time.sleep(0.5)
            search_box.send_keys(Keys.ENTER)
            
            # 검색 결과 로딩 대기
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(1)

            # 검색 결과 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            data = soup.find('div', 'c-pageSiteSearch-results')
            if not data:
                print(f"검색 결과 없음: {game}")
                success = True  # 검색 결과가 없는 것은 성공으로 처리
                continue

            text_tag = data.find('p', 'g-text-medium-fluid g-text-bold g-outer-spacing-bottom-small u-text-overflow-ellipsis')
            if not text_tag:
                print(f"게임 이름 태그 없음: {game}")
                success = True  # 태그가 없는 것은 성공으로 처리
                continue

            text = text_tag.text.strip()
            clean_target = clean_game_name(game)
            clean_found = clean_game_name(text)

            if clean_target != clean_found:
                print(f"검색어와 불일치: {game} → {text}")
                success = True  # 불일치는 성공으로 처리
                continue

            print(f"{game} 와 '{text}' 은 같다고 판단")

            # 상세 페이지 이동
            link_tag = data.find('a', href=True)
            href_value = link_tag['href']
            url1 = f'https://www.metacritic.com{href_value}'
            driver.get(url1)
            
            # 페이지 로딩 대기
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            score = soup.find('div', class_='c-productHero_scoreInfo g-inner-spacing-top-medium g-outer-spacing-bottom-medium g-outer-spacing-top-medium')

            if score:
                score_all = score.find_all('div', class_='c-productScoreInfo_scoreNumber u-float-right')
                meta = [s.text.strip() for s in score_all]
                meta_score = meta[0] if len(meta) > 0 else 'N/A'
                user_score = meta[1] if len(meta) > 1 else 'N/A'
            else:
                meta_score = 'N/A'
                user_score = 'N/A'

            # 결과 저장
            df_saved = pd.concat([
                df_saved,
                pd.DataFrame([{
                    "검색어": game,
                    "게임명": text,
                    "메타스코어": meta_score,
                    "유저스코어": user_score,
                    "게임URL": url1
                }])
            ], ignore_index=True)

            # 중간 저장
            df_saved.to_csv(save_file, index=False, encoding="utf-8-sig")
            print(f"저장 완료: {game}")
            success = True

        except TimeoutException as e:
            retry_count += 1
            print(f"타임아웃 에러 발생 ({retry_count}/{max_retries}): {game}")
            print(f"에러 내용: {e}")
            
            if retry_count < max_retries:
                print("새로운 브라우저로 재시도...")
                driver = restart_driver(driver)
            else:
                print(f"최대 재시도 횟수 초과 → 건너뜀: {game}")
                
        except WebDriverException as e:
            retry_count += 1
            print(f"브라우저 오류 발생 ({retry_count}/{max_retries}): {game}")
            print(f"에러 내용: {e}")
            
            if retry_count < max_retries:
                print("새로운 브라우저로 재시도...")
                driver = restart_driver(driver)
            else:
                print(f"최대 재시도 횟수 초과 → 건너뜀: {game}")
                
        except Exception as e:
            retry_count += 1
            print(f"예상치 못한 오류 발생 ({retry_count}/{max_retries}): {game}")
            print(f"에러 내용: {e}")
            
            if retry_count < max_retries:
                print("새로운 브라우저로 재시도...")
                driver = restart_driver(driver)
            else:
                print(f"최대 재시도 횟수 초과 → 건너뜀: {game}")

print("=== 모든 게임 처리 완료 ===")

# 마지막에 브라우저 종료
try:
    driver.quit()
    print("브라우저 종료 완료")
except:
    print("브라우저 종료 중 오류 발생")

이미 처리된 게임 수: 3803
이미 처리된 게임 → 건너뜀: GUILTY GEAR -STRIVE-
이미 처리된 게임 → 건너뜀: Magic: The Gathering Arena
검색 시작: EZ2ON REBOOT : R
검색어와 불일치: EZ2ON REBOOT : R → Reboot (2022)
이미 처리된 게임 → 건너뜀: Sons Of The Forest
이미 처리된 게임 → 건너뜀: Left 4 Dead 2
이미 처리된 게임 → 건너뜀: Human Fall Flat
이미 처리된 게임 → 건너뜀: Riders Republic
이미 처리된 게임 → 건너뜀: Phasmophobia
이미 처리된 게임 → 건너뜀: Company of Heroes 3
이미 처리된 게임 → 건너뜀: Granblue Fantasy Versus: Rising
이미 처리된 게임 → 건너뜀: Tinkerlands
이미 처리된 게임 → 건너뜀: Dota 2
이미 처리된 게임 → 건너뜀: Crime Simulator
이미 처리된 게임 → 건너뜀: Backrooms: Escape Together
이미 처리된 게임 → 건너뜀: Warhammer 40,000: Darktide
이미 처리된 게임 → 건너뜀: World of Tanks
이미 처리된 게임 → 건너뜀: Satisfactory
이미 처리된 게임 → 건너뜀: Unrailed!
이미 처리된 게임 → 건너뜀: Monster Hunter: World
이미 처리된 게임 → 건너뜀: NARAKA: BLADEPOINT
이미 처리된 게임 → 건너뜀: Liar's Bar
이미 처리된 게임 → 건너뜀: A Way Out
검색 시작: The Binding of Isaac: Repentance
검색창 못 찾음 → 새 창 열어서 재시도: The Binding of Isaac: Repentance
검색어와 불일치: The Binding of Isaac: Repentance → The Binding of Isaac: Rebirth
이미 처리된 게임 → 건너뜀: Ne

In [2]:
asd = pd.read_csv('metacritic_scores.csv')
asd

,게임명,meta_score,user_score,검색어,메타스코어,유저스코어,게임URL
0,GUILTY GEAR -STRIVE-,87,7.9,NaN,NaN,NaN,NaN
1,Magic: The Gathering Arena,84,3.1,NaN,NaN,NaN,NaN
2,Sons Of The Forest,86,7.3,NaN,NaN,NaN,NaN
3,Left 4 Dead 2,89,8.2,NaN,NaN,NaN,NaN
4,Human Fall Flat,70,7.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15541,Strange Block 36,NaN,NaN,Strange Block 36,tbd,tbd,https://www.metacritic.com/game/strange-block-36/
15542,Raft,NaN,NaN,Raft,tbd,7.4,https://www.metacritic.com/game/raft/
15543,Chasing Tails -A Promise in the Snow-,NaN,NaN,Chasing Tails ~A Promise in the Snow~,tbd,7.0,https://www.metacritic.com/game/chasing-tails-...
15544,ARK: Survival Evolved,NaN,NaN,ARK: Survival Evolved,70,6.5,https://www.metacritic.com/game/ark-survival-e...


In [5]:
asd.drop(['meta_score', 'user_score', '검색어', '게임URL'], axis=1, inplace=True)

In [9]:
asd[['메타스코어', '유저스코어']] = asd[['메타스코어', '유저스코어']].fillna(0)

In [13]:
asd['메타스코어'].value_counts()

메타스코어
tbd    12361
0       1292
74        90
76        84
75        84
       ...  
31         1
35         1
39         1
17         1
28         1
Name: count, Length: 73, dtype: int64

In [14]:
asd['유저스코어'].value_counts()

유저스코어
tbd    12080
0       1292
7.5       89
7.8       85
7.6       81
       ...  
9.1        1
0.3        1
9.8        1
2.1        1
2.9        1
Name: count, Length: 91, dtype: int64

In [15]:
asd['메타스코어'] = asd['메타스코어'].replace('tbd', '0')
asd['유저스코어'] = asd['유저스코어'].replace('tbd', '0')

In [18]:
asd.to_csv('최종_메타크리틱점수.csv', index=False)